<a href="https://colab.research.google.com/github/IlTACK-OH/pandas_practice/blob/main/day4/08_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 자료형 다루기

In [8]:
import pandas as pd
import seaborn as sns

tips = sns.load_dataset('tips')

자료형을 변환하려면 `astype`메서드를 사용하면 된다.<br>
`astype`메서드를 사용하여  `sex`열의 데이터를 문자열로 변환하여 `sex_str`이라는 새로운 열에 저장한다.

In [9]:
tips['sex_str'] = tips['sex'].astype(str)

In [10]:
print(tips.dtypes)

total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
sex_str         object
dtype: object


자료형이 변환된 데이터를 다시 원래대로 만들수도 있다.

In [11]:
tips['total_bill'] = tips['total_bill'].astype(str)
print(tips.dtypes)

total_bill      object
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
sex_str         object
dtype: object


`total_bill` 열을 다시 실수로 변환하겠다.

In [12]:
tips['total_bill'] = tips['total_bill'].astype(float)
print(tips.dtypes)

total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
sex_str         object
dtype: object


# 잘못 입력한 문자열 처리하기

잘못 입력한 데이터를 변환하는 방법에 대해서 알아보겠다.<br>이런 경우 자료형을 변환하는 `to_numeric`메서드도 함께 알아보겠다.

In [16]:
tips_sub_miss = tips.head(10).copy() # 책대로 했을 시 warning message출력
tips_sub_miss.loc[[1,3,5,7],'total_bill'] = 'missing'
tips_sub_miss

,total_bill,tip,sex,smoker,day,time,size,sex_str
0,16.99,1.01,Female,No,Sun,Dinner,2,Female
1,missing,1.66,Male,No,Sun,Dinner,3,Male
2,21.01,3.50,Male,No,Sun,Dinner,3,Male
3,missing,3.31,Male,No,Sun,Dinner,2,Male
4,24.59,3.61,Female,No,Sun,Dinner,4,Female
5,missing,4.71,Male,No,Sun,Dinner,4,Male
6,8.77,2.00,Male,No,Sun,Dinner,2,Male
7,missing,3.12,Male,No,Sun,Dinner,4,Male
8,15.04,1.96,Male,No,Sun,Dinner,2,Male
9,14.78,3.23,Male,No,Sun,Dinner,2,Male


---
책의 그대로를 실행하면 SettingWithCopyWarning을 반환한다.<br> 이는 pandas가 데이터프레임을 메모리에 할당하는 방식과 연관이 있다.<br> 자세한 사항은 아래의 사이트를 참고하면 좋다.
>참고사이트
https://emilkwak.github.io/pandas-dataframe-settingwithcopywarning

---
데이터프레임의 자료형을 확인해 보면 `totall_bill`열이 실수가 아니라 문자열임을 알 수 있다.<br>
`missing`이라는 문자열 때문에 이런 문제가 발생한 것이다.



In [17]:
tips_sub_miss.dtypes

total_bill      object
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
sex_str         object
dtype: object

`astype`메서드를 사용하면 이 문제를 해결할 수 있을까?<br>
그 답은 **"없다"**이다.<br>
판다스는 'missing'이라는 문자열을 실수로 변환하는 방법을 모르기 때문이다.

In [18]:
tips_sub_miss['total_bill'].astype(float)

ValueError: ignored

따라서 이럴 때 사용하는 방법이 `to_numeric`메서드이다.

In [19]:
pd.to_numeric(tips_sub_miss['total_bill'])

ValueError: ignored

**구라다.**<br>
사실 `to_numeric`메서드를 사용해도 문자열을 실수로 변환할 수 없다.<br><br>
하지만 `to_numeric`메서드는 `errors`인자에 `raise`,`coerce`,`ignore`를 지정하여 오류를 어느 정도 제어할 수 있다.
>**erros 인자에 설정할 수 있는 값**
- raise: 숫자로 변환할 수 없는 값이 있으면 오류 발생
- coerce: 숫자로 변환할 수 없는 값을 누락값으로 지정
- ignore: 아무 작업도 안함.

In [21]:
# erros = ignoer
tips_sub_miss['total_bill'] = pd.to_numeric(tips_sub_miss['total_bill'], errors='ignore')
tips_sub_miss.dtypes

total_bill      object
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
sex_str         object
dtype: object

`total_bill`은 여전히 문자열이다.

In [24]:
# erros = coerce
tips_sub_miss['total_bill'] = pd.to_numeric(tips_sub_miss['total_bill'], errors='coerce')
tips_sub_miss.dtypes

total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
sex_str         object
dtype: object

'missing'값이 누락값으로 바뀌어, total_bill의 자료형이 실수로 바뀐다.

`to_numeric`메서드에는 `errors`인자 외에도 **`downcast`**인자가 있다.<br> `downcast`는 정수, 실수와 같은 자료형을 더 작은 형태로 만들 때 사용한다. 이를 다운캐스트라고 한다.<br>
`downcast`인자에는 `integer`,`signed`,`unsigned`,`float` 등의 값을 사용할 수 있다. 

In [25]:
tips_sub_miss['total_bill'] = pd.to_numeric(tips_sub_miss['total_bill'],errors='coerce',downcast='float')
tips_sub_miss.dtypes

total_bill     float32
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
sex_str         object
dtype: object

`float64`는 `float32`보다 더 많은 범위의 실수를 표현할 수 있지만 메모리 공간을 2배나 차지한다. 만약 저장하는 실수의 예상 범위가 크지 않다면 다운캐스트 하는 것이 좋다.

# 카테고리 자료형
판다스 라이브러리는 유한한 범위의 값만을 가질 수 있는 카테고리라는 특수한 자료형이 있다.<br>
>**카테고리 자료형의 장점과 특징**
- 용량과 속도 면에서 매우 효율적이다.
- 주로 동일한 문자열이 반복되어 데이터를 구성하는 경우에 사용한다.

In [26]:
tips['sex'] = tips['sex'].astype('str')
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    object  
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
 7   sex_str     244 non-null    object  
dtypes: category(3), float64(2), int64(1), object(2)
memory usage: 10.8+ KB


데이터프레임 용량이 10.7KB 정도라는 것을 확인할 수 있다.

In [27]:
tips['sex']= tips['sex'].astype('category')
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
 7   sex_str     244 non-null    object  
dtypes: category(4), float64(2), int64(1), object(1)
memory usage: 9.3+ KB


데이터프레임의 용량이 9.3KB정도로 줄어든 것을 알 수 있다.<br>
이와 같이 반복되는 문자열로 구성된 데이터는 카테고리를 사용하는 것이 더 효율적이다.